In [1]:
# Fetch and organize data
import os, zipfile, glob, io, requests

# Tabular data
import numpy as np

import pandas as pd
import geopandas as gpd
from osgeo import gdal
import georasters as gr

# Plots
import matplotlib.pyplot as plt


# Useful Functions

Clipping shapefiles (to clip rasters)

In [2]:
# importing the data
world_df = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# subseting columns
world_df = world_df[['continent','name', 'iso_a3','geometry']]

# croppping boundaries and save shapefiles
path = os.path.join(os.getcwd(), 'shapefiles')
if os.path.isdir(path) == False:
    os.mkdir(path) # create folder

for i in ['Colombia', 'Belize']:
    temp = world_df.loc[world_df.name == i, ].reset_index(drop=True)
    temp.to_file("./shapefiles/" + i.lower() + ".shp")

# load the shapefiles from cropping
colombia_shp = './shapefiles/colombia.shp'
belize_shp = './shapefiles/belize.shp'

For fetching raster from Hansen

In [3]:
# Fetch hansen datasets
def fetch_raster(list_url, country_name):
    ''' Fetcht the rasters by from a list url, download it and 
    subset 2 rasters:
    - url : raster dataset url from Hansen
    - country_name: to keep separated folders by country name'''

    # main_folder
    current_folder = os.getcwd()
    new_folder = str(country_name)
    
    # arrangements
    path = os.path.join(current_folder, new_folder)
    if os.path.isdir(path) == False:
        os.mkdir(path) # create folder
        
    os.chdir(path) # move to folder

    # Fetching geotif form Hansen
    for url in list_url:
        fetching_url = f'! wget {url} -q'
        if os.path.exists(url.rsplit('/')[-1]) == False:
            os.system(fetching_url)

    # print the outcome
    print("================================================================================")
    print('%2d raster files downloaded in folder: %s.' %(len(list_url), new_folder))

    # return to working directory
    os.chdir(current_folder)

    print("================================================================================")

Clip raster using shapefile as borderline

In [4]:
# Function to clip and save rasters
def clip_raster(merged_raster, inshp, country_name, raster_type):
    ''' Get a clipped raster from the Worldclim data for specified countries:
    - merged_raster : geotiff merged under the country borderline
    - inshp : path clipped shapefile with the name of the country
    - country_name: country name of the raster/folder for clipped outraster 
    '''

    path = os.path.join(os.getcwd(), 'hansen_' + country_name)
    if os.path.isdir(path) == False:
        os.mkdir(path) # create folder

    outraster = path + '/' + str(country_name) + '_' + str(raster_type) + '.tif'
    dsClip = gdal.Warp(outraster, merged_raster, cutlineDSName=inshp, cropToCutline=True, dstNodata=np.nan)
  
    # print the results
    print('%s clipped raster succesfully stored at the folder: hansen_%s.' %(raster_type, country_name))
    dsClip= None
    print("================================================================================")

Merge and clip raster

In [7]:
def merge_clip(country_name, country_shp, raster_type):
    ''' merge_clip recieve a country_name folder, merge the raster inside and clipped based on
    another country_shp shapefile path, finally create a folder with country_name with
    a hanse_ prefix and store the clipped data.
    
    :params country_name: string country name in lowercase
    :params country_shp: borderline shapefile path
    :params raster_type: string raster type downloaded

    :returns: a message with the path of the stored clipped raster
    '''
    
    # Getting path
    path = os.path.join(os.getcwd(), country_name)

    # Read the data
    files = glob.glob(os.path.join(path, '*tif')) # colect all the files in folder "country_name"

    # Merging the rasters
    inputs = ' '.join(files)
    output = os.path.join(path,  country_name + '-merged.tif')

    # Command
    command = "gdal_merge.py %s -o %s" % (inputs, output)

    # Run the command
    print("Running gdal wrap:\n")
    os.system(command) # 0 execute a succesful merge
    print("================================================================================")

    # Clipping the rasters
    print("Running gdal clipped")
    clip_raster(gdal.Open(output), country_shp, country_name, raster_type)

    # removing the folder
    command_remove = "rm -r %s" %(path)
    print("%s folder removed" %(country_name))
    os.system(command_remove)
    print("================================================================================")

# **TREE COVER**

## **Tree cover in Colombia**

In [6]:
# Colombia treecover data
colombia_treecover = [
    'https://storage.googleapis.com/earthenginepartners-hansen/GFC-2019-v1.7/Hansen_GFC-2019-v1.7_treecover2000_10N_080W.tif',
    'https://storage.googleapis.com/earthenginepartners-hansen/GFC-2019-v1.7/Hansen_GFC-2019-v1.7_treecover2000_20N_080W.tif',
    'https://storage.googleapis.com/earthenginepartners-hansen/GFC-2019-v1.7/Hansen_GFC-2019-v1.7_treecover2000_10N_070W.tif',
    'https://storage.googleapis.com/earthenginepartners-hansen/GFC-2019-v1.7/Hansen_GFC-2019-v1.7_treecover2000_00N_080W.tif',
    'https://storage.googleapis.com/earthenginepartners-hansen/GFC-2019-v1.7/Hansen_GFC-2019-v1.7_treecover2000_00N_070W.tif',
    'https://storage.googleapis.com/earthenginepartners-hansen/GFC-2019-v1.7/Hansen_GFC-2019-v1.7_treecover2000_20N_070W.tif'
    ]

# Fetching datasets
fetch_raster(colombia_treecover, 'colombia')

# Merge and clip raster
merge_clip('colombia', colombia_shp, 'treecover')

 6 raster files downloaded in folder: colombia.
Running gdal wrap:

0...10...20...30...40...50...60...70...80...90...100 - done.
Running gdal clipped


Warning 1: for band 1, destination nodata value has been rounded to 0, Byte being an integer datatype.


treecover clipped raster succesfully stored at the folder: hansen_colombia.
colombia folder removed


### **Tree cover in Belize**

In [8]:
# Belize treecover data
belize_treecover = ['https://storage.googleapis.com/earthenginepartners-hansen/GFC-2019-v1.7/Hansen_GFC-2019-v1.7_treecover2000_20N_090W.tif']

# Fetching datasets
fetch_raster(belize_treecover, 'belize')

# Merge and clip raster
merge_clip('belize', belize_shp, 'treecover')

 1 raster files downloaded in folder: belize.
Running gdal wrap:

0...10...20...30...40...50...60...70...80...90...100 - done.
Running gdal clipped


Warning 1: for band 1, destination nodata value has been rounded to 0, Byte being an integer datatype.


treecover clipped raster succesfully stored at the folder: hansen_belize.
belize folder removed


In case a review is necessary:

In [ ]:
# # Read the data
# files = glob.glob(os.path.join(path, '*tif')) # list of files in folder
# names = [i[i.rfind('treecover2000'):].rsplit('.')[0] for i in files] # name for read data
# colombia =#  list(zip(names, [gdal.Open(f) for f in files]))

# # Review
# for i in colombia:
#     name, raster = i
#     print('File: ', name)

#     # Projection and Raster Metadata
#     # print(raster.GetProjection(), raster.GetMetadata())

#     # Dimensions X, Y and Number of bands
#     print(raster.RasterXSize, raster.RasterYSize, raster.RasterCount)
#     print()

# Projection and Raster Metadata
# print(merged.GetProjection(), merged.GetMetadata())

# Dimensions X, Y and Number of bands
# print(merged.RasterXSize, merged.RasterYSize, merged.RasterCount)
# print()